所需要运行环境：
1. AWS中的p2.xlarge
2. 安装chainer, 方法如下：<br>
      git clone https://github.com/chainer/chainercv <br>
      cd chainercv <br>
      conda env create -f environment.yml <br>
      source activate chainercv <br>
      pip install -e . <br>
3. train set与test set所在的路径为:
   /home/ubuntu/distracted_driver_detection/imgs/train <br>
   /home/ubuntu/distracted_driver_detection/imgs/test <br>
4. 需要新建一个存放test_roi的文件夹：<br>
   mkdir /home/ubuntu/distracted_driver_detection/imgs/test_roi

In [1]:
# In the rest of the tutorial, we assume that the `plt`
# is imported before every code snippet.
import matplotlib.pyplot as plt

import chainer
chainer.cuda.get_device_from_id(0).use()

from chainercv.datasets import voc_bbox_label_names
from chainercv.links import SSD300, FasterRCNNVGG16
from chainercv.utils import read_image
from chainercv.visualizations import vis_bbox

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

model = SSD300(pretrained_model='voc0712')
model.to_gpu()

2018-08-02 08:18:39,702 [INFO]: Lock 140339064425808 acquired on /home/ubuntu/.chainer/dataset/_dl_cache/_dl_lock
2018-08-02 08:18:39,704 [INFO]: Lock 140339064425808 released on /home/ubuntu/.chainer/dataset/_dl_cache/_dl_lock


From: https://chainercv-models.preferred.jp/ssd300_voc0712_converted_2017_06_06.npz
To: /home/ubuntu/.chainer/dataset/_dl_cache/7580b6888d3f7742a1b531234009c0b0
  %   Total    Recv       Speed  Time left
100   93MiB   93MiB   5635KiB/s    0:00:00

2018-08-02 08:18:56,736 [INFO]: Lock 140339064425808 acquired on /home/ubuntu/.chainer/dataset/_dl_cache/_dl_lock
2018-08-02 08:18:56,739 [INFO]: Lock 140339064425808 released on /home/ubuntu/.chainer/dataset/_dl_cache/_dl_lock
/home/ubuntu/anaconda3/envs/chainer_p27/lib/python2.7/site-packages/chainercv/utils/link.py:60: RuntimeWarning: cv2 is not installed on your environment. Pretrained models are trained with cv2. The performace may change with Pillow backend.
  RuntimeWarning)


In [1]:
import os, sys
import urllib2
import datetime
from PIL import Image
from shutil import copyfile

import pandas as pd
import numpy as np

def gererate_trainset_roi():
    
    csv_path = '/home/ubuntu/distracted_driver_detection/driver_imgs_list.csv'
    PERSON = 14

    imgs = pd.read_csv(csv_path)
    count = imgs.shape[0]

    print("Start time: ")
    print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    
    for i in range(0, count):    
        #get the image file
        print "i = ", i
        bb = []
        driver_status = imgs.iloc[i]['classname']
        image = imgs.iloc[i]['img']
        imgs_path = os.path.join('/home/ubuntu/distracted_driver_detection/imgs/train', 
                             driver_status, image)
        
        f = image.split(".")
        roi_file_path = os.path.join('/home/ubuntu/distracted_driver_detection/imgs/train', 
                                     driver_status, f[0]+"_"+"ROI"+"."+f[1])
    
        img = read_image(imgs_path)
        bboxes, labels, scores = model.predict([img])
    
        # the object number of Detector >= 2, and person can be detected
        if len(labels[0]) >= 2 and PERSON in labels[0]:
            print "labels[0] = ", labels[0]
            person_dict = {PERSON:[]}

            loc = 0
            for value in labels[0]:
                if value == PERSON:
                    person_dict[PERSON].append(loc)
                loc += 1
            
            index = 0
            prob_index_dict = {}
            for prob in scores[0]:
                if index in person_dict[PERSON]:
                    prob_index_dict[index] = prob
                index += 1
        
            #Get the index for max probility of driver
            index = sorted(prob_index_dict.items(), key=lambda item:item[1], reverse=True)[0][0]
            bb = bboxes[0][index]

        # the object number of Detector = 1, and person can be detected
        elif len(labels[0]) == 1 and PERSON in labels[0]:
            bb = bboxes[0][0]
        
        #person cannot be detected by SSD300
        else:        
            vis_bbox(img, bboxes[0], labels[0], scores[0], label_names=voc_bbox_label_names)
            plt.show()
            print("Driver cannot be detected by SSD300")
            print(imgs_path)
            print(roi_file_path)
            copyfile(imgs_path, roi_file_path)
            continue  
    
        x, y = bb[1], bb[0]
        height = bb[2] - bb[0]
        if len(labels[0]) >= 2:
            width = 680 - bb[1]
        else:
            width = bb[3] - bb[1]    

        #get the ROI image from img file
        img = Image.open(imgs_path)
        ROI = img.crop((x, y, width, height))
    
        try:
            ROI.save(roi_file_path)
        except SystemError:
            print('System Error')
            copyfile(imgs_path, roi_file_path)
    
        if i % 200 == 0:
            print "i = ", i
            print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        
    print("End time:")
    print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    
#gererate_trainset_roi()

In [ ]:
def gererate_testset_roi():
    
    print("Start time: ")
    print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    
    path = os.path.join('/home/ubuntu/distracted_driver_detection/imgs',
                        'test', '*.jpg')
    files = glob.glob(path)

    i = 0
    count = len(files)
    for fl in files:
        
        #if i < value:
        #    i += 1
        #    continue
        print i
        i += 1
        
        filename = os.path.basename(fl)
        f = filename.split(".")
        roi_file_path = os.path.join('/home/ubuntu/distracted_driver_detection/imgs/test_roi', 
                                     f[0]+"_"+"ROI"+"."+f[1])
        
        img = read_image(fl)
        bboxes, labels, scores = model.predict([img])        
    
        # the object number of Detector >= 2
        if len(labels[0]) >= 2 and PERSON in labels[0]:
            print "labels[0] = ", labels[0]
            person_dict = {PERSON:[]}

            loc = 0
            for value in labels[0]:
                if value == PERSON:
                    person_dict[PERSON].append(loc)
                loc += 1
            
            index = 0
            prob_index_dict = {}
            for prob in scores[0]:
                if index in person_dict[PERSON]:
                    prob_index_dict[index] = prob
                index += 1
        
            index = sorted(prob_index_dict.items(), key=lambda item:item[1], reverse=True)[0][0]
            bb = bboxes[0][index]
            
        elif len(labels[0]) == 1 and PERSON in labels[0]:
            bb = bboxes[0][0]
            
        else:
            vis_bbox(img, bboxes[0], labels[0], scores[0], label_names=voc_bbox_label_names)
            plt.show()
            print("Driver cannot be detected by SSD300")
            print(imgs_path)
            print(roi_file_path)
            copyfile(imgs_path, roi_file_path)
            continue          
    
        x, y = bb[1], bb[0]
        height = bb[2] - bb[0]
        if len(labels[0]) >= 2:
            width = 680 - bb[1]
        else:
            width = bb[3] - bb[1]    

        img = Image.open(imgs_path)
        ROI = img.crop((x, y, width, height))

        try:
            ROI.save(roi_file_path)
        except SystemError:
            print('System Error')
            copyfile(imgs_path, roi_file_path)
    
        if i % 200 == 0:
            print "i = ", i
            print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        
    print("End time:")
    print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))        

#gererate_testset_roi()